In [ ]:
%load_ext autoreload

# 引入必要的库

In [ ]:
import os

checkboard_data=['/home/lyh/data/cameraCal']
key_list = []

print('---------------------------------------------------')
for sampleset in checkboard_data:
    print('---->',sampleset,'有：',len(os.listdir(sampleset)),'个样本')
    for e in os.listdir(sampleset):
        print(e)
        key_list.append(e)

key_list = ['cal5']

# 棋盘格设置
 - (9,6）说明9列6行，不包含最外层一圈
 - objp 是棋盘格三维空间点
 - imgpoints是2D坐标

In [ ]:
import numpy as np
CHECKERBOARD_SIZE = (9, 6)
objp = np.zeros((CHECKERBOARD_SIZE[0] * CHECKERBOARD_SIZE[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:CHECKERBOARD_SIZE[0], 0:CHECKERBOARD_SIZE[1]].T.reshape(-1, 2)
objpoints = []
imgpoints = []

# 读取并标定

In [ ]:
%autoreload
import sys,os
import numpy as np
import torch
import torch.nn.functional as F
import cv2
import math,time
import matplotlib.pyplot as plt

from tianmoucv.isp import fourdirection2xy,poisson_blend
from tianmoucv.data import TianmoucDataReader

for key in key_list:
    dataset = TianmoucDataReader(checkboard_data,
                                 MAXLEN=1000,
                                 matchkey=key,
                                 speedUpRate=1)
    for index in range(20,len(dataset)-20,10):
        sample = dataset[index]
        F0 = sample['F0']
        #print('shapes:',F0.shape)

        F0 = (F0*255).astype(np.uint8)

        gray = cv2.cvtColor(F0, cv2.COLOR_RGB2GRAY)

        # 获取角点：靠近点拍比较容易检测
        ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD_SIZE, None)
        
        if ret:
            objpoints.append(objp)
            # Refine corners for greater accuracy
            criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.01)
            corners_refined = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria )
            imgpoints.append(corners_refined)

        
            print(index)
            print('----->success:',len(imgpoints))


In [ ]:

# Calibrate camera
ret, camera_matrix, distortion_coeff, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

# Save calibration parameters as a file
np.savez('calibration_parameters.npz', camera_matrix=camera_matrix, distortion_coeff=distortion_coeff)

print(camera_matrix)

# f/dx, 0, u
# 0 f/dy,  v
# 1,  1,   1